In [1]:
import os
from pyprojroot import here
os.environ["HF_HOME"] = str(here("cache/huggingface"))
import lightning as L
from importlib import reload
import sys
sys.path.append(str(here()))
from models import model_definitions as MD
import torch
import pandas as pd
reload(MD)
torch.set_float32_matmul_precision('medium')
data_module_3 = MD.GoEmotionsDataModule(pd.read_parquet(here("data/goemotions_3.parquet")))
data_module_7 = MD.GoEmotionsDataModule(pd.read_parquet(here("data/goemotions_7.parquet")))
data_module_28 = MD.GoEmotionsDataModule(pd.read_parquet(here("data/goemotions_28.parquet")))

for module in [data_module_3, data_module_7, data_module_28]:
  module.prepare_data()
  module.setup()

test_df_3 = data_module_3.test_df
test_df_7 = data_module_7.test_df
test_df_28 = data_module_28.test_df

trainer = L.Trainer(
  default_root_dir=here("cache/lightning"),
  enable_checkpointing=True,
  fast_dev_run=False,
  accelerator="cpu", 
)

trainer

Seed set to 1
Seed set to 1
Seed set to 1
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\Plancha\emotion-recognition\.pixi\envs\default\Lib\site-packages\lightning\pytorch\trainer\connectors\logger_connector\logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [2]:
pd.set_option("display.max_colwidth", 100)
pd.set_option('display.max_rows',20)
top20 = test_df_28[["text", "emotions"]].head(20)
top20

,text,emotions
11258,"For some reason I'm imagining a hauty, French snorting laugh.","[[desire], [amusement], [amusement]]"
23191,I believe the mentally ill are the ones that spend so much time and effort caring about what I d...,"[[caring], [optimism], [annoyance], [unclear], [unclear]]"
10956,"I dont watch battle rap. Do these guys come up with these on the spot, because that's amazing.","[[admiration], [confusion], [admiration]]"
4650,I hate Twitter more and more each day....the people on there are so toxic it literally makes me ...,"[[anger], [anger], [disappointment, sadness]]"
15818,I can't post on r/dankmemes :(,"[[disapproval], [unclear], [disapproval]]"
32846,I can't see anything wrong with this message,"[[approval, optimism], [disapproval], [approval], [unclear], [approval]]"
23313,Both directions? They should all be arrested. This will cost the people of Edmonton a lot of tim...,"[[annoyance], [annoyance], [anger]]"
14133,Someone tell [NAME] to report on these threats to journalists and the first amendment!,"[[unclear], [unclear], [relief]]"
13214,"In case you're wondering, Nintendo does put a bittering agent in their Switch cartridges...and i...","[[fear], [annoyance], [fear, surprise], [disgust], [annoyance, disappointment]]"
12159,Sorry for your loss yesterday. Your team is poised to do even better in the years to come and ho...,"[[gratitude], [optimism, remorse], [optimism, sadness]]"


In [3]:
reload(MD)
distil_bert_wmse_28 = MD.DistilBertFinetuneOnWeightedMSE.load_from_checkpoint(
  here(r"cache\lightning\lightning_logs\wmse_28\checkpoints\epoch=12-step=7977.ckpt"),
  n_emotions=28
)
distil_bert_wmse_28

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertFinetuneOnWeightedMSE(
  (model): DistilBertForSequenceClassification(
    (distilbert): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): DistilBertSdpaAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_features=768, bias=True)
            )
            (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=T

In [4]:
import numpy as np
def out_to_emotion(out, df):
  """Print model outputs to a df"""
  # out is a list of lists of probabilities for each emotion
  ret = pd.DataFrame(
    np.array([i.numpy() for i in out]).reshape(-1, 28),
    # columns where they start with emotion_
    columns = df.columns[df.columns.str.startswith("emotion_")],
    index = df.index
  )
  # add the text column and put it first
  return ret
out = trainer.predict(
  distil_bert_wmse_28,
  test_df_28.head(20)["text"].tolist()
)
outs = out_to_emotion(out, test_df_28.head(20))
outs

Predicting: |          | 0/? [00:00<?, ?it/s]

,emotion_sadness,emotion_unclear,emotion_love,emotion_gratitude,emotion_disapproval,emotion_amusement,emotion_disappointment,emotion_disgust,emotion_admiration,emotion_realization,...,emotion_approval,emotion_nervousness,emotion_embarrassment,emotion_surprise,emotion_anger,emotion_grief,emotion_pride,emotion_desire,emotion_relief,emotion_fear
11258,0.014639,0.195887,0.002003,0.001547,0.031562,0.262273,0.021087,0.046404,0.019945,0.023929,...,0.051151,0.010977,0.025925,0.192177,0.013892,0.000517,0.006855,0.011604,0.006378,0.042916
23191,0.162931,0.199867,0.128191,0.004986,0.206556,0.001804,0.075796,0.004739,0.005769,0.115060,...,0.103075,0.009070,0.000829,0.003113,0.018684,0.000129,0.000500,0.017614,0.000257,0.040061
10956,0.001639,0.156398,0.045308,0.000517,0.166092,0.049477,0.010344,0.003264,0.582488,0.017890,...,0.134183,0.002427,0.001452,0.027933,0.001563,0.000038,0.004025,0.028160,0.001320,0.006639
4650,0.037505,0.061428,0.063807,0.001805,0.120586,0.010199,0.066470,0.205742,0.008597,0.023707,...,0.006615,0.021324,0.009297,0.006395,0.111662,0.000762,0.000311,0.026767,0.000109,0.045444
15818,0.125190,0.640002,0.000909,0.001460,0.058288,0.017355,0.079123,0.002568,0.004144,0.039645,...,0.058054,0.002156,0.002648,0.016197,0.005994,0.000581,0.000228,0.012095,0.000586,0.012371
32846,0.014731,0.192943,0.014705,0.001813,0.253929,0.001037,0.070314,0.028276,0.011165,0.051141,...,0.122960,0.007484,0.001373,0.025448,0.024034,0.000400,0.000316,0.044631,0.002322,0.016430
23313,0.008013,0.293982,0.000399,0.006999,0.100785,0.001290,0.047435,0.064153,0.018223,0.027390,...,0.091664,0.003598,0.006107,0.006611,0.533507,0.001991,0.001915,0.002028,0.001921,0.005898
14133,0.003731,0.635456,0.000343,0.000696,0.039364,0.001033,0.020544,0.012353,0.002880,0.035161,...,0.132766,0.006304,0.018885,0.045037,0.240517,0.002833,0.005781,0.005083,0.001467,0.056360
13214,0.041190,0.399363,0.004173,0.001096,0.244459,0.003418,0.065363,0.107754,0.013017,0.040980,...,0.057550,0.007462,0.022967,0.018001,0.020654,0.000299,0.000049,0.003591,0.000309,0.085413
12159,0.060973,0.036624,0.014913,0.033723,0.025860,0.000190,0.071871,0.004211,0.238365,0.032897,...,0.104843,0.004311,0.001917,0.026484,0.004888,0.012150,0.055139,0.011196,0.016421,0.018995


In [5]:
# join top20 with outs
top20_p = top20.join(outs)
top20_p

,text,emotions,emotion_sadness,emotion_unclear,emotion_love,emotion_gratitude,emotion_disapproval,emotion_amusement,emotion_disappointment,emotion_disgust,...,emotion_approval,emotion_nervousness,emotion_embarrassment,emotion_surprise,emotion_anger,emotion_grief,emotion_pride,emotion_desire,emotion_relief,emotion_fear
11258,"For some reason I'm imagining a hauty, French snorting laugh.","[[desire], [amusement], [amusement]]",0.014639,0.195887,0.002003,0.001547,0.031562,0.262273,0.021087,0.046404,...,0.051151,0.010977,0.025925,0.192177,0.013892,0.000517,0.006855,0.011604,0.006378,0.042916
23191,I believe the mentally ill are the ones that spend so much time and effort caring about what I d...,"[[caring], [optimism], [annoyance], [unclear], [unclear]]",0.162931,0.199867,0.128191,0.004986,0.206556,0.001804,0.075796,0.004739,...,0.103075,0.009070,0.000829,0.003113,0.018684,0.000129,0.000500,0.017614,0.000257,0.040061
10956,"I dont watch battle rap. Do these guys come up with these on the spot, because that's amazing.","[[admiration], [confusion], [admiration]]",0.001639,0.156398,0.045308,0.000517,0.166092,0.049477,0.010344,0.003264,...,0.134183,0.002427,0.001452,0.027933,0.001563,0.000038,0.004025,0.028160,0.001320,0.006639
4650,I hate Twitter more and more each day....the people on there are so toxic it literally makes me ...,"[[anger], [anger], [disappointment, sadness]]",0.037505,0.061428,0.063807,0.001805,0.120586,0.010199,0.066470,0.205742,...,0.006615,0.021324,0.009297,0.006395,0.111662,0.000762,0.000311,0.026767,0.000109,0.045444
15818,I can't post on r/dankmemes :(,"[[disapproval], [unclear], [disapproval]]",0.125190,0.640002,0.000909,0.001460,0.058288,0.017355,0.079123,0.002568,...,0.058054,0.002156,0.002648,0.016197,0.005994,0.000581,0.000228,0.012095,0.000586,0.012371
32846,I can't see anything wrong with this message,"[[approval, optimism], [disapproval], [approval], [unclear], [approval]]",0.014731,0.192943,0.014705,0.001813,0.253929,0.001037,0.070314,0.028276,...,0.122960,0.007484,0.001373,0.025448,0.024034,0.000400,0.000316,0.044631,0.002322,0.016430
23313,Both directions? They should all be arrested. This will cost the people of Edmonton a lot of tim...,"[[annoyance], [annoyance], [anger]]",0.008013,0.293982,0.000399,0.006999,0.100785,0.001290,0.047435,0.064153,...,0.091664,0.003598,0.006107,0.006611,0.533507,0.001991,0.001915,0.002028,0.001921,0.005898
14133,Someone tell [NAME] to report on these threats to journalists and the first amendment!,"[[unclear], [unclear], [relief]]",0.003731,0.635456,0.000343,0.000696,0.039364,0.001033,0.020544,0.012353,...,0.132766,0.006304,0.018885,0.045037,0.240517,0.002833,0.005781,0.005083,0.001467,0.056360
13214,"In case you're wondering, Nintendo does put a bittering agent in their Switch cartridges...and i...","[[fear], [annoyance], [fear, surprise], [disgust], [annoyance, disappointment]]",0.041190,0.399363,0.004173,0.001096,0.244459,0.003418,0.065363,0.107754,...,0.057550,0.007462,0.022967,0.018001,0.020654,0.000299,0.000049,0.003591,0.000309,0.085413
12159,Sorry for your loss yesterday. Your team is poised to do even better in the years to come and ho...,"[[gratitude], [optimism, remorse], [optimism, sadness]]",0.060973,0.036624,0.014913,0.033723,0.025860,0.000190,0.071871,0.004211,...,0.104843,0.004311,0.001917,0.026484,0.004888,0.012150,0.055139,0.011196,0.016421,0.018995


In [6]:
# per text, print most probable emotions and real emotions
for i, row in top20_p.iterrows():
  print(f"Text: {row['text']}")
  print("Predicted emotions:")
  emotions = row[2:].sort_values(ascending=False).head(3)
  for emotion, prob in emotions.items():
    print(f"{emotion}: {prob:.4f}")
  print("Real emotions:")
  print(str([i.tolist() for i in row['emotions']]))
  print("\n")

Text: For some reason I'm imagining a hauty, French snorting laugh.
Predicted emotions:
emotion_joy: 0.2756
emotion_amusement: 0.2623
emotion_unclear: 0.1959
Real emotions:
[['desire'], ['amusement'], ['amusement']]


Text: I believe the mentally ill are the ones that spend so much time and effort caring about what I do with my own time and money. ¯_(ツ)_/¯
Predicted emotions:
emotion_disapproval: 0.2066
emotion_unclear: 0.1999
emotion_sadness: 0.1629
Real emotions:
[['caring'], ['optimism'], ['annoyance'], ['unclear'], ['unclear']]


Text: I dont watch battle rap. Do these guys come up with these on the spot, because that's amazing.
Predicted emotions:
emotion_admiration: 0.5825
emotion_disapproval: 0.1661
emotion_unclear: 0.1564
Real emotions:
[['admiration'], ['confusion'], ['admiration']]


Text: I hate Twitter more and more each day....the people on there are so toxic it literally makes me sick. They can't appreciate a good meme.
Predicted emotions:
emotion_disgust: 0.2057
emotion_

In [7]:
str([i.tolist() for i in row['emotions']])

"[['disapproval'], ['unclear'], ['disapproval'], ['curiosity', 'desire'], ['annoyance']]"